# Notebook Summary

The purpose of this notebook is to implement a trading strategy where you're buying/selling dogecoin based on sentiment from Elon Musk's twitter feed 


# Save Your Credentials in Your .env File

##  Alpaca

Make sure you have a .env file set up with your Alpaca API keys. The notebook is looking for your Alpaca API keys in this format: 

API_KEY = "xxx"

API_SECRET = "xxx"

## Twitter

Link to Twitter Developer API Portal: https://developer.twitter.com/en

Make sure you also store your credentials for the twitter developer api in your .env file. 
This notebook assumes that you have "essential" access (please note that you can request a higher access level)

The additional Twitter credentials you need to store in the .env file are the following: 

TWITTER_API_KEY = "XXX"

TWITTER_API_KEY = "XXX"

TWITTER_API_SECRET_KEY = "XXX"

TWITTER_BEARER_TOKEN = "XXX"

TWITTER_ACCESS_TOKEN = "XXX"

TWITTER_ACCESS_TOKEN_SECRET = "XXX"

In [34]:
import pandas as pd
from twarc import Twarc2, expansions
import json
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
import datetime
import tweepy
import requests
from dotenv import load_dotenv

#Twitter API Github Examples Found Here:
## https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/6a-labs-code-academic-python.md

In [35]:
# Load the environment variables from the .env file
load_dotenv()

True

# Connect to Alpaca Paper Trading API

In [36]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets" #notice "PAPER" in the URL

In [37]:
# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

# Create client for Twitter API

In [5]:
# grab twitter keys, following instructions from github: 

#client = Twarc2(bearer_token=os.getenv("TWITTER_BEARER_TOKEN"))
TWITTER_API_KEY = os.getenv("TWITTER_API_KEY")
TWITTER_API_SECRET_KEY = os.getenv("TWITTER_API_SECRET_KEY")
TWITTER_BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

In [6]:
tweepy.__version__

'4.10.1'

In [7]:
client = tweepy.Client( bearer_token=TWITTER_BEARER_TOKEN, 
                        consumer_key=TWITTER_API_KEY, 
                        consumer_secret=TWITTER_API_SECRET_KEY, 
                        access_token=TWITTER_ACCESS_TOKEN, 
                        access_token_secret=TWITTER_ACCESS_TOKEN_SECRET, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

# Find recent tweets from Elon Musk:

Right now, API returns a max of 100 tweets over the past 7 days. API access needs to be upgraded from "essential" in order to go back more days. 
Other fields besides 'author_id' and 'created_at' can be returned as well.


In [8]:
#https://www.kirenz.com/post/2021-12-10-twitter-api-v2-tweepy-and-pandas-in-python/twitter-api-v2-tweepy-and-pandas-in-python/

# Define query
query = 'from:elonmusk'# -is:retweet'

# get max. 100 tweets
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'], #there are more fields we could grab
                                     max_results=100)

In [9]:
# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data) 
df.head()

,text,id,created_at,author_id
0,@PopMech Why is this a PopMech story?,1573026765295030273,2022-09-22T19:09:40.000Z,44196397
1,@BillyM2k https://t.co/vFVHaB1l4t https://t.co...,1573007878042296320,2022-09-22T17:54:37.000Z,44196397
2,@ajtourville @Twitter @paraga Yup (sigh),1573004188791066627,2022-09-22T17:39:58.000Z,44196397
3,@MuskUniversity Fully reusable orbital rockets...,1572986371295289351,2022-09-22T16:29:10.000Z,44196397
4,@stocktalkweekly @WholeMarsBlog @SawyerMerritt...,1572985952683065345,2022-09-22T16:27:30.000Z,44196397


In [10]:
#Create pandas date column

df["Date"] = df["created_at"].str[:10]
#df["Date"] = df["created_at"].dt.tz_localize(None)
df["Date"] = pd.to_datetime(df['Date'])
df.head()

,text,id,created_at,author_id,Date
0,@PopMech Why is this a PopMech story?,1573026765295030273,2022-09-22T19:09:40.000Z,44196397,2022-09-22
1,@BillyM2k https://t.co/vFVHaB1l4t https://t.co...,1573007878042296320,2022-09-22T17:54:37.000Z,44196397,2022-09-22
2,@ajtourville @Twitter @paraga Yup (sigh),1573004188791066627,2022-09-22T17:39:58.000Z,44196397,2022-09-22
3,@MuskUniversity Fully reusable orbital rockets...,1572986371295289351,2022-09-22T16:29:10.000Z,44196397,2022-09-22
4,@stocktalkweekly @WholeMarsBlog @SawyerMerritt...,1572985952683065345,2022-09-22T16:27:30.000Z,44196397,2022-09-22


In [14]:
#Create pandas time column

df["Time"] = df["created_at"].str[11:19]
#df["Date"] = df["created_at"].dt.tz_localize(None)
df["Time"] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df.head()

,text,id,created_at,author_id,Date,Time
0,@PopMech Why is this a PopMech story?,1573026765295030273,2022-09-22T19:09:40.000Z,44196397,2022-09-22,19:09:40
1,@BillyM2k https://t.co/vFVHaB1l4t https://t.co...,1573007878042296320,2022-09-22T17:54:37.000Z,44196397,2022-09-22,17:54:37
2,@ajtourville @Twitter @paraga Yup (sigh),1573004188791066627,2022-09-22T17:39:58.000Z,44196397,2022-09-22,17:39:58
3,@MuskUniversity Fully reusable orbital rockets...,1572986371295289351,2022-09-22T16:29:10.000Z,44196397,2022-09-22,16:29:10
4,@stocktalkweekly @WholeMarsBlog @SawyerMerritt...,1572985952683065345,2022-09-22T16:27:30.000Z,44196397,2022-09-22,16:27:30


In [15]:
# Filter to dates of interest
# User to specify dates
# May want to specify times later

df = df.set_index("Date")

tweet_start_date = "2022-09-21"
#tweet_start_date = pd.to_datetime(tweet_start_date)  #datetime.strptime(tweet_start_date, '%Y-%m-%d')

tweet_end_date = "2022-09-22"
#tweet_end_date = pd.to_datetime(tweet_end_date)

df_tweets_fromtimeframe = df.loc[tweet_start_date:tweet_end_date]
display(df_tweets_fromtimeframe.head())
display(df_tweets_fromtimeframe.tail())

,text,id,created_at,author_id,Time
Date,,,,,
2022-09-22,@PopMech Why is this a PopMech story?,1573026765295030273,2022-09-22T19:09:40.000Z,44196397,19:09:40
2022-09-22,@BillyM2k https://t.co/vFVHaB1l4t https://t.co...,1573007878042296320,2022-09-22T17:54:37.000Z,44196397,17:54:37
2022-09-22,@ajtourville @Twitter @paraga Yup (sigh),1573004188791066627,2022-09-22T17:39:58.000Z,44196397,17:39:58
2022-09-22,@MuskUniversity Fully reusable orbital rockets...,1572986371295289351,2022-09-22T16:29:10.000Z,44196397,16:29:10
2022-09-22,@stocktalkweekly @WholeMarsBlog @SawyerMerritt...,1572985952683065345,2022-09-22T16:27:30.000Z,44196397,16:27:30


,text,id,created_at,author_id,Time
Date,,,,,
2022-09-21,@teslaownersSV @RBReich Reich is suicidally bo...,1572415723846406151,2022-09-21T02:41:37.000Z,44196397,02:41:37
2022-09-21,@TrungTPhan 🤣,1572415147049623552,2022-09-21T02:39:19.000Z,44196397,02:39:19
2022-09-21,@RBReich You both an idiot and a liar,1572414455681814528,2022-09-21T02:36:34.000Z,44196397,02:36:34
2022-09-21,@BillyM2k 🤣,1572396931866918914,2022-09-21T01:26:56.000Z,44196397,01:26:56
2022-09-21,@ajtourville @parag @Twitter Curiouser and cur...,1572396742267609090,2022-09-21T01:26:11.000Z,44196397,01:26:11


# Run Twitter Sentiment Analysis on Text & Buy/Sell Dogecoin Depending on Sentiment

Algorithms below place market orders

## Text sentiment analysis (update later)

In [26]:
# simple example: if Elon Musk talks about dogecoin, buy dogecoin: 

doge_terms = ["DOGE"]
doge_qty = 1000

df["text_upper"] = df["text"].str.upper()
df_doge = df.loc[df["text_upper"].str.contains('|'.join(doge_terms))]
display(df_doge)

if len(df_doge) > 0:
    print("\nElon Musk talked about Dogecoin recently")
    print(f"buy dogecoin with alpaca trading api, default value of {doge_qty} coins. Update quantity if needed")

,text,id,created_at,author_id,Time,text_upper
Date,,,,,,
2022-09-21,@cb_doge @PPathole @RBReich 🤣,1572567768607916033,2022-09-21T12:45:47.000Z,44196397,12:45:47,@CB_DOGE @PPATHOLE @RBREICH 🤣
2022-09-18,@cb_doge 🤣,1571345862575460354,2022-09-18T03:50:22.000Z,44196397,03:50:22,@CB_DOGE 🤣
2022-09-17,@LarckeningXuruo @cb_doge It will be later thi...,1571148447444783105,2022-09-17T14:45:55.000Z,44196397,14:45:55,@LARCKENINGXURUO @CB_DOGE IT WILL BE LATER THI...



Elon Musk talked about Dogecoin recently
buy dogecoin with alpaca trading api, default value of 1000 coins. Update quantity if needed


## Determine whether to buy/sell Dogecoin

In [27]:
buy_dogecoin = False

if len(df_doge) > 0:
    print("Elon Musk talked about Dogecoin recently")
    print("buy dogecoin")
    buy_dogecoin = True

Elon Musk talked about Dogecoin recently
buy dogecoin


## Place trades with Alpaca trading API

As of the time of writing the quantity of 1000 dogecoins corresponds to approx $60 USD worth

In [ ]:
if buy_dogecoin == True:
    api.submit_order('DOGEUSD', qty=1000, side='buy', time_in_force="gtc")

## Make sure your order when through (open orders list should be empty)

In [ ]:
# list all of your open orders: 

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

open_orders_list = api.list_orders(status='open')
open_orders_list

## Get details about the order that you recently placed

In [ ]:
# print closed orders

closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

closed_orders = [o for o in closed_orders if o.symbol == 'DOGEUSD']
print(closed_orders)